In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os
os.getcwd()
os.chdir('/content/gdrive/MyDrive/Dogs_vs_Cats')

In [ ]:
os.listdir()

['test.zip',
 'train.zip',
 'hdf5',
 '.ipynb_checkpoints',
 'output',
 'test',
 'train',
 'results_finetuning_dogs_vs_cats2.csv',
 'results_finetuning_dogs_vs_cats3.csv',
 'results_finetuning_dogs_vs_cats4.csv',
 'dogs_vs_cats1.model',
 'dogs_vs_cats.model']

In [ ]:
# import the necessary packages
import h5py
import os
class HDF5DatasetWriter:
	def __init__(self, dims, outputPath, dataKey="images", bufSize=1000):
		# check to see if the output path exists, and if so, raise
		# an exception
		if os.path.exists(outputPath):
			raise ValueError("The supplied ‘outputPath‘ already "
				"exists and cannot be overwritten. Manually delete "
				"the file before continuing.", outputPath)

		# open the HDF5 database for writing and create two datasets:
		# one to store the images/features and another to store the
		# class labels
		self.db = h5py.File(outputPath, "w")
		self.data = self.db.create_dataset(dataKey, dims, 
			dtype="float")
		self.labels = self.db.create_dataset("labels", (dims[0],),
			dtype="int")

		# store the buffer size, then initialize the buffer itself
		# along with the index into the datasets
		self.bufSize = bufSize
		self.buffer = {"data": [], "labels": []}
		self.idx = 0

	def add(self, rows, labels):
		# add the rows and labels to the buffer
		self.buffer["data"].extend(rows)
		self.buffer["labels"].extend(labels)

		# check to see if the buffer needs to be flushed to disk
		if len(self.buffer["data"]) >= self.bufSize:
			self.flush()

	def flush(self):
		# write the buffers to disk then reset the buffer
		i = self.idx + len(self.buffer["data"])
		self.data[self.idx:i] = self.buffer["data"]
		self.labels[self.idx:i] = self.buffer["labels"]
		self.idx = i
		self.buffer = {"data": [], "labels": []}

	def storeClassLabels(self, classLabels):
		# create a dataset to store the actual class label names,
		# then store the class labels
		dt = h5py.special_dtype(vlen=str)
		labelSet = self.db.create_dataset("label_names",
		(len(classLabels),), dtype=dt)
		labelSet[:] = classLabels

	def close(self):
		# check to see if there are any other entries in the buffer
		# that need to be flushed to disk
		if len(self.buffer["data"]) > 0:
			self.flush()
		# close the dataset
		self.db.close()

In [ ]:
# !unzip test.zip -d test

In [ ]:
# !unzip train.zip

In [ ]:

# define the paths to the images directory
IMAGES_PATH = r"/content/gdrive/MyDrive/Dogs_vs_Cats/train"
TEST_IMAGES_PATH = r"/content/gdrive/MyDrive/Dogs_vs_Cats/test"

# since we do not have validation data or access to the testing
# labels we need to take a number of images from the training
# data and use them instead

NUM_CLASSES = 2
NUM_VAL_IMAGES = 2500 * NUM_CLASSES
# NUM_TEST_IMAGES = 1250 * NUM_CLASSES

# define the path to the output training, validation, and testing
# HDF5 files
TRAIN_HDF5 = r"/content/gdrive/MyDrive/Dogs_vs_Cats/hdf5/train.hdf5"
VAL_HDF5 = r"/content/gdrive/MyDrive/Dogs_vs_Cats/hdf5/val.hdf5"
TEST_HDF5 = r"/content/gdrive/MyDrive/Dogs_vs_Cats/hdf5/test.hdf5"

# path to the output model file
MODEL_PATH = r"/content/gdrive/MyDrive/Dogs_vs_Cats/dogs_vs_cats.model"

# define the path to the dataset mean
DATASET_MEAN = r"/content/gdrive/MyDrive/Dogs_vs_Cats/output/dogs_vs_cats_mean.json"

# define the path to the output directory used for storing plots,
# classification reports, etc.
OUTPUT_PATH = r"/content/gdrive/MyDrive/Dogs_vs_Cats/output"

INIT_LR = 1e-4
bs = 32
NUM_EPOCHS = 10

In [ ]:
len(os.listdir('/content/gdrive/MyDrive/Dogs_vs_Cats/train'))

25000

In [ ]:
import re

def sorted_nicely( l ): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from imutils import paths
import numpy as np
import progressbar
import json
import cv2
import os


trainPaths = list(paths.list_images(IMAGES_PATH))
trainLabels = [os.path.basename(label).split('.')[0] for label in trainPaths]
print(trainLabels[0:10])

testPaths = list(paths.list_images(TEST_IMAGES_PATH))
testPaths = sorted_nicely(testPaths)
print(testPaths[0:10])
testLabels = [os.path.basename(label).split('.')[0] for label in testPaths]
print(testLabels[0:10])

le = LabelEncoder()
trainLabels = le.fit_transform(trainLabels)

split = train_test_split(trainPaths, trainLabels, test_size=NUM_VAL_IMAGES, stratify=trainLabels, random_state=42)
(trainPaths, valPaths, trainLabels, valLabels) = split

['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat']
['/content/gdrive/MyDrive/Dogs_vs_Cats/test/1.jpg', '/content/gdrive/MyDrive/Dogs_vs_Cats/test/2.jpg', '/content/gdrive/MyDrive/Dogs_vs_Cats/test/3.jpg', '/content/gdrive/MyDrive/Dogs_vs_Cats/test/4.jpg', '/content/gdrive/MyDrive/Dogs_vs_Cats/test/5.jpg', '/content/gdrive/MyDrive/Dogs_vs_Cats/test/6.jpg', '/content/gdrive/MyDrive/Dogs_vs_Cats/test/7.jpg', '/content/gdrive/MyDrive/Dogs_vs_Cats/test/8.jpg', '/content/gdrive/MyDrive/Dogs_vs_Cats/test/9.jpg', '/content/gdrive/MyDrive/Dogs_vs_Cats/test/10.jpg']
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']


In [ ]:
datasets = [
("train", trainPaths, trainLabels, TRAIN_HDF5),
("val", valPaths, valLabels, VAL_HDF5),
("test", testPaths, testLabels, TEST_HDF5)
]

for (dType, paths, labels, outputPath) in datasets:
	
	print("[INFO] building {}...".format(outputPath))
	dataset = HDF5DatasetWriter((len(paths), 256, 256, 3), outputPath)

	# initialize the progress bar
	# initialize the progress bar
	widgets = ["Extracting Features: ", progressbar.Percentage(), " ", 
	progressbar.Bar(), " ", progressbar.ETA()]
	pbar = progressbar.ProgressBar(maxval=len(paths), 
		widgets=widgets).start()

	# loop over the images in batches
	for i in np.arange(0, len(paths), bs):
		# extract the batch of images and labels, then initialize the
		# list of actual images that will be passed through the network
		# for feature extraction
		batchPaths = paths[i:i + bs]
		batchLabels = labels[i:i + bs]
		batchImages = []
		# loop over the images and labels in the current batch
		for (j, imagePath) in enumerate(batchPaths):
			# load the input image using the Keras helper utility
			# while ensuring the image is resized to 224x224 pixels
			image = load_img(imagePath, target_size=(256, 256))
			image = img_to_array(image)
			
			# preprocess the image by (1) expanding the dimensions and
			# (2) subtracting the mean RGB pixel intensity from the
			# ImageNet dataset
			image = np.expand_dims(image, axis=0)
			image = imagenet_utils.preprocess_input(image)
			
			# add the image to the batch
			batchImages.append(image)
		
		# pass the images through the network and use the outputs as
		# our actual features
		batchImages = np.vstack(batchImages)
		print(batchImages.shape)
		# add the features and labels to our HDF5 dataset
		dataset.add(batchImages, batchLabels)
		pbar.update(i)
			
	# close the dataset
	dataset.close()
	pbar.finish()

Extracting Features: N/A% |                                    | ETA:  --:--:--

[INFO] building /content/gdrive/MyDrive/Dogs_vs_Cats/hdf5/train.hdf5...


Extracting Features: N/A% |                                    | ETA:  --:--:--

(32, 256, 256, 3)


Extracting Features:   0% |                                    | ETA:   2:46:33

...

Extracting Features:  99% |################################### | ETA:   0:00:11

(32, 256, 256, 3)


Extracting Features:  99% |################################### | ETA:   0:00:02

(8, 256, 256, 3)


Extracting Features: 100% |####################################| Time:  0:23:45


In [ ]:
import h5py
db = h5py.File(TRAIN_HDF5)
print(list(db.keys()))
print(db["images"].shape)
print(db["labels"].shape)
print()

db = h5py.File(VAL_HDF5)
print(list(db.keys()))
print(db["images"].shape)
print(db["labels"].shape)
print()

db = h5py.File(TEST_HDF5)
print(list(db.keys()))
print(db["images"].shape)
print(db["labels"].shape)
print()

['images', 'labels']
(20000, 256, 256, 3)
(20000,)

['images', 'labels']
(5000, 256, 256, 3)
(5000,)

['images', 'labels']
(12500, 256, 256, 3)
(12500,)



In [ ]:
# drive.flush_and_unmount()

In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report

baseModel = ResNet50(weights="imagenet", include_top=False, 
                     input_tensor=Input(shape=(224, 224, 3)))
# baseModel = ResNet50(weights="imagenet", include_top=False)

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)
model.summary()

In [ ]:
from keras.utils import np_utils
import numpy as np
import h5py

class HDF5DatasetGenerator:

	def __init__(self, dbPath, batchSize, preprocessors=None,
		aug=None, binarize=True, classes=2):
		# store the batch size, preprocessors, and data augmentor,
		# whether or not the labels should be binarized, along with
		# the total number of classes
		self.batchSize = batchSize
		self.preprocessors = preprocessors
		self.aug = aug
		self.binarize = binarize
		self.classes = classes

		# open the HDF5 database for reading and determine the total
		# number of entries in the database
		self.db = h5py.File(dbPath)
		self.numImages = self.db["labels"].shape[0]

	def generator(self, passes=np.inf):
		epochs=0
		while epochs<passes:
			for i in np.arange(0, self.numImages, self.batchSize):
				images = self.db["images"][i:i+self.batchSize]
				labels = self.db["labels"][i:i+self.batchSize]

				if self.binarize:
					labels = np_utils.to_categorical(labels, self.classes)

				if self.preprocessors is not None:
					procImages = []
					for image in images:
						for p in self.preprocessors:
							image = p.preprocess(image)
						procImages.append(image)
					images = np.array(procImages)

				if self.aug is not None:
					(images, labels) = next(self.aug.flow(images, labels, batch_size=self.batchSize))
				yield(images, labels)
			epochs = epochs+1


	def close(self):
		self.db.close()

In [ ]:
import cv2

class SimplePreprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
	# store the target image width, height, and interpolation
	# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter

	def preprocess(self, image):
	# resize the image to a fixed size, ignoring the aspect
	# ratio
		return cv2.resize(image, (self.width, self.height), interpolation=self.inter)

In [ ]:
from sklearn.feature_extraction.image import extract_patches_2d

class PatchPreprocessor:
	def __init__(self, width, height):
		self.width = width
		self.height = height

	def preprocess(self, image):
		return extract_patches_2d(image, (self.height, self.width), max_patches=1)[0]

In [ ]:
# import the necessary packages
from keras.preprocessing.image import img_to_array
	
class ImageToArrayPreprocessor:
	def __init__(self, dataFormat=None):
		# store the image data format
		self.dataFormat = dataFormat
	
	def preprocess(self, image):
		# apply the Keras utility function that correctly rearranges
		# the dimensions of the image
		return img_to_array(image, data_format=self.dataFormat)

In [ ]:
# initialize the image preprocessors
sp = SimplePreprocessor(224, 224)
pp = PatchPreprocessor(224, 224)
iap = ImageToArrayPreprocessor()

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, 
                         width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, 
                         horizontal_flip=True, fill_mode="nearest")

# initialize the training and validation dataset generators
trainGen = HDF5DatasetGenerator(TRAIN_HDF5, 128, aug=aug, preprocessors = [pp, iap], classes=2)
print(trainGen.numImages)
valGen = HDF5DatasetGenerator(VAL_HDF5, 128, preprocessors = [sp, iap], classes=2)
print(valGen.numImages)
testGen = HDF5DatasetGenerator(TEST_HDF5, 50, classes=2, binarize=False)
print(testGen.numImages)

checkpoint = ModelCheckpoint(MODEL_PATH, monitor="val_loss", mode="min", 
                             save_best_only=True, verbose=1)
callbacks = [checkpoint]

20000
5000
12500


In [ ]:
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
from keras.optimizers import Adam

# compile the model
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / NUM_EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, 
              metrics=["accuracy"])

In [ ]:
# train the network
H = model.fit(
	trainGen.generator(),
	steps_per_epoch=trainGen.numImages // 128,
	validation_data=valGen.generator(),
	validation_steps=valGen.numImages // 128,
	epochs=NUM_EPOCHS,
	max_queue_size=128 * 2,
	callbacks=callbacks, verbose=1)

Epoch 1/10
156/156 [==============================] - 4332s 28s/step - loss: 0.1685 - accuracy: 0.9428 - val_loss: 0.0481 - val_accuracy: 0.9846

Epoch 00001: val_loss improved from inf to 0.04809, saving model to /content/gdrive/MyDrive/Dogs_vs_Cats/dogs_vs_cats.model


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Dogs_vs_Cats/dogs_vs_cats.model/assets
Epoch 2/10
156/156 [==============================] - 4167s 27s/step - loss: 0.0676 - accuracy: 0.9778 - val_loss: 0.0376 - val_accuracy: 0.9876

Epoch 00002: val_loss improved from 0.04809 to 0.03756, saving model to /content/gdrive/MyDrive/Dogs_vs_Cats/dogs_vs_cats.model


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Dogs_vs_Cats/dogs_vs_cats.model/assets
Epoch 3/10
156/156 [==============================] - 4161s 27s/step - loss: 0.0547 - accuracy: 0.9808 - val_loss: 0.0332 - val_accuracy: 0.9886

Epoch 00003: val_loss improved from 0.03756 to 0.03323, saving model to /content/gdrive/MyDrive/Dogs_vs_Cats/dogs_vs_cats.model


...

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Dogs_vs_Cats/dogs_vs_cats.model/assets
Epoch 4/10
156/156 [==============================] - 4152s 27s/step - loss: 0.0475 - accuracy: 0.9833 - val_loss: 0.0330 - val_accuracy: 0.9882

Epoch 00004: val_loss improved from 0.03323 to 0.03299, saving model to /content/gdrive/MyDrive/Dogs_vs_Cats/dogs_vs_cats.model


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Dogs_vs_Cats/dogs_vs_cats.model/assets
Epoch 5/10
156/156 [==============================] - 4130s 27s/step - loss: 0.0475 - accuracy: 0.9831 - val_loss: 0.0307 - val_accuracy: 0.9900

Epoch 00005: val_loss improved from 0.03299 to 0.03066, saving model to /content/gdrive/MyDrive/Dogs_vs_Cats/dogs_vs_cats.model


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Dogs_vs_Cats/dogs_vs_cats.model/assets
Epoch 6/10
148/156 [===========================>..] - ETA: 2:51 - loss: 0.0464 - accuracy: 0.9830

In [ ]:
import matplotlib.pyplot as plt

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, NUM_EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, NUM_EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

In [ ]:
# save the model to file
# close the HDF5 datasets
trainGen.close()
valGen.close()

In [ ]:
print("[INFO] evaluating network...")
predictions = model.predict(testGen.generator(), 
                            steps=testGen.numImages // 50, max_queue_size=50 * 2)

[INFO] evaluating network...


In [ ]:
pred = list(predictions.argmax(axis=1))
print(len(predictions))
print(predictions)

12500
[[5.6938179e-09 1.0000000e+00]
 [3.0976742e-11 1.0000000e+00]
 [3.5042418e-09 1.0000000e+00]
 ...
 [1.4418547e-08 1.0000000e+00]
 [7.3487932e-10 1.0000000e+00]
 [1.0000000e+00 2.9002339e-10]]


In [ ]:
testGen.close()

In [ ]:
pred[0:10]

[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]

In [ ]:
pred_prob_dog = list(predictions[:,1])

In [ ]:
index = [i+1 for i in range (12500)]

import pandas as pd
df = pd.DataFrame(columns=['id', 'label'])

df['id'] = index
df['label'] = pred_prob_dog
df.head(15)

,id,label
0,1,1.000000e+00
1,2,1.000000e+00
2,3,1.000000e+00
3,4,1.000000e+00
4,5,1.882441e-09
5,6,6.470183e-07
6,7,3.186987e-09
7,8,2.304098e-09
8,9,1.270717e-09
9,10,1.369617e-06


In [ ]:
df.to_csv('results_finetuning_dogs_vs_cats2.csv', index=False)

In [ ]:
testGen = HDF5DatasetGenerator(TEST_HDF5, 50, classes=2, binarize=False)
print(testGen.numImages)

12500


In [ ]:
from keras.models import load_model
print("[INFO] loading model...")
model = load_model(MODEL_PATH)

[INFO] loading model...


In [ ]:
import numpy as np
import cv2

class CropPreprocessor:
	def __init__(self, width, height, horiz=True, inter=cv2.INTER_AREA):
		self.width = width
		self.height = height
		self.horiz = horiz
		self.inter = inter

	def preprocess(self, image):
		crops = []
		(h, w) = image.shape[:2]
		coords = [
		[0, 0, self.width, self.height],
		[w - self.width, 0, w, self.height],
		[w - self.width, h - self.height, w, h],
		[0, h - self.height, self.width, h]
    ]
		# compute the center crop of the image as well
		dW = int(0.5 * (w - self.width))
		dH = int(0.5 * (h - self.height))
		coords.append([dW, dH, w - dW, h - dH])

		for (startX, startY, endX, endY) in coords:
			crop = image[startY:endY, startX:endX]
			crop = cv2.resize(crop, (self.width, self.height), interpolation=self.inter)
			crops.append(crop)

		if self.horiz:
			mirrors = [cv2.flip(c,1) for c in crops]
			crops.extend(mirrors)

		return(np.array(crops))

In [ ]:
cp = CropPreprocessor(224, 224)

In [ ]:
# initialize the progress bar
widgets = ["Evaluating: ", progressbar.Percentage(), " ",
progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=testGen.numImages // 50,
widgets=widgets).start()

Evaluating: N/A% |                                             | ETA:  --:--:--

In [ ]:
predictions = []
# loop over a single pass of the test data
for (i, (images, labels)) in enumerate(testGen.generator(passes=1)):
	# loop over each of the individual images
	for image in images:
		# apply the crop preprocessor to the image to generate 10
		# separate crops, then convert them from images to arrays
		crops = cp.preprocess(image)
		crops = np.array([iap.preprocess(c) for c in crops],
		dtype="float32")

		# make predictions on the crops and then average them
		# together to obtain the final prediction
		pred = model.predict(crops)
		predictions.append(pred.mean(axis=0))

	# update the progress bar
	pbar.update(i)
pbar.finish()

Evaluating: 100% |#############################################| Time:  5:37:04


In [ ]:
l = []
for p in predictions:
  l.append(list(p))
print(l)

[[5.934181e-10, 1.0], [1.6844873e-12, 1.0], [2.1646804e-09, 1.0], [2.1387856e-07, 0.9999998], [1.0, 2.68223e-08], [0.99999964, 3.69068e-07], [1.0, 1.0380744e-09], [1.0, 5.028188e-09], [0.9999997, 2.5374626e-07], [0.99999505, 5.0193e-06], [0.99999964, 3.4088026e-07], [2.3512042e-05, 0.99997646], [1.0, 1.4216805e-08], [0.9993305, 0.00066941715], [1.0, 5.1193314e-08], [0.99999696, 3.0006063e-06], [0.00506162, 0.9949384], [3.6586234e-11, 1.0], [1.0, 3.1295766e-09], [0.99999875, 1.2616431e-06], [1.548801e-11, 1.0], [1.0, 1.016003e-08], [1.3898441e-10, 1.0], [4.1904258e-07, 0.99999964], [0.9999995, 4.8520064e-07], [1.2319183e-05, 0.9999877], [4.3760583e-08, 1.0], [1.0, 3.2287033e-09], [0.7984797, 0.20152032], [1.650235e-07, 0.9999998], [1.571625e-10, 1.0], [0.5152637, 0.4847363], [1.2236898e-08, 1.0], [1.0, 6.2272667e-09], [1.0, 1.1721572e-10], [0.9999999, 9.67711e-08], [1.0, 1.2103598e-09], [1.0, 8.093034e-10], [0.002467922, 0.99753207], [1.0, 5.7447608e-08], [3.84427e-08, 1.0], [2.2445183e

In [ ]:
pred = np.array(l).argmax(axis=1)

In [ ]:
np.array(l)

array([[5.9341809e-10, 1.0000000e+00],
       [1.6844873e-12, 1.0000000e+00],
       [2.1646804e-09, 1.0000000e+00],
       ...,
       [6.3386846e-10, 1.0000000e+00],
       [9.6501765e-12, 1.0000000e+00],
       [1.0000000e+00, 8.9608071e-11]], dtype=float32)

In [ ]:
pred[0:10]

array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0])

In [ ]:
pred_prob_dog = list(np.array(l)[:,1])
pred_prob_dog

[1.0,
 1.0,
 1.0,
 0.9999998,
 2.68223e-08,
 3.69068e-07,
 1.0380744e-09,
 5.028188e-09,
 2.5374626e-07,
 5.0193e-06,
 3.4088026e-07,
 0.99997646,
 1.4216805e-08,
 0.00066941715,
 5.1193314e-08,
 3.0006063e-06,
 0.9949384,
 1.0,
 3.1295766e-09,
 1.2616431e-06,
 1.0,
 1.016003e-08,
 1.0,
 0.99999964,
 4.8520064e-07,
 0.9999877,
 1.0,
 3.2287033e-09,
 0.20152032,
 0.9999998,
 1.0,
 0.4847363,
 1.0,
 6.2272667e-09,
 1.1721572e-10,
 9.67711e-08,
 1.2103598e-09,
 8.093034e-10,
 0.99753207,
 5.7447608e-08,
 1.0,
 1.0,
 1.0,
 0.9999998,
 3.6562167e-10,
 0.992241,
 9.747397e-09,
 0.9999998,
 1.0,
 5.0054904e-08,
 1.1231103e-09,
 2.4326525e-07,
 1.4825896e-05,
 2.0216248e-07,
 2.7879707e-07,
 0.99999106,
 1.0,
 1.3253688e-10,
 0.99999887,
 5.7158798e-09,
 5.7344623e-10,
 0.99999774,
 1.458609e-10,
 2.7084523e-05,
 1.0,
 0.99531996,
 1.0,
 2.855991e-09,
 1.0,
 1.0,
 0.99999934,
 1.0,
 1.0,
 1.0,
 0.0001613503,
 0.9999998,
 1.0,
 0.99999887,
 0.9999992,
 3.085739e-06,
 0.01277071,
 9.66215e-06,
 

In [ ]:
index = [i+1 for i in range (12500)]

import pandas as pd
df = pd.DataFrame(columns=['id', 'label'])

df['id'] = index
df['label'] = pred_prob_dog
df.head(15)

,id,label
0,1,1.000000e+00
1,2,1.000000e+00
2,3,1.000000e+00
3,4,9.999998e-01
4,5,2.682230e-08
5,6,3.690680e-07
6,7,1.038074e-09
7,8,5.028188e-09
8,9,2.537463e-07
9,10,5.019300e-06


In [ ]:
df.to_csv('results_finetuning_dogs_vs_cats5.csv', index=False)

In [ ]:
testGen.close()